# 0. Initialisation

In [1]:
import pandas as pd
import numpy as np 
from helpers2 import S3Connection

In [2]:
s3 = S3Connection(bucket_name="clichere/diffusion")

Connection successful


# 1. Ouverture des fichiers

## 1.1. DPE après juillet 2021

In [3]:
path_logements_existants = "DPE/DPE_ADEME/dpe-v2-logements-existants.csv"
DPE_logements_existants = s3.read_csv_from_s3(path_logements_existants)

In [4]:
path_logements_neufs = "DPE/DPE_ADEME/dpe-v2-logements-neufs.csv"
DPE_logements_neufs = s3.read_csv_from_s3(path_logements_neufs)

In [6]:
print(len(DPE_logements_existants.columns))
print(len(DPE_logements_neufs.columns))

33
30


In [8]:
set(DPE_logements_existants.columns) - set(DPE_logements_neufs.columns)

{'Année_construction', 'Méthode_application_DPE', 'Période_construction'}

In [10]:
print(DPE_logements_existants.columns)

Index(['Date_réception_DPE', 'Date_établissement_DPE', 'Modèle_DPE',
       'Date_fin_validité_DPE', 'Version_DPE', 'Méthode_application_DPE',
       'Etiquette_DPE', 'Etiquette_GES', 'Année_construction', 'Type_bâtiment',
       'Période_construction', 'Surface_habitable_logement', 'Adresse_brute',
       'Nom__commune_(BAN)', 'Code_INSEE_(BAN)', 'N°_voie_(BAN)',
       'Identifiant__BAN', 'Adresse_(BAN)', 'Code_postal_(BAN)', 'Score_BAN',
       'Nom__rue_(BAN)', 'Coordonnée_cartographique_X_(BAN)',
       'Coordonnée_cartographique_Y_(BAN)', 'Code_postal_(brut)',
       'N°_étage_appartement', 'Nom_résidence', 'Cage_d'escalier',
       'Complément_d'adresse_logement', 'Statut_géocodage',
       'Nom__commune_(Brut)', 'N°_département_(BAN)', 'N°_région_(BAN)',
       'Complément_d'adresse_bâtiment'],
      dtype='object')


Il y a trois variables en plus dans le fichier logements : Année_construction, Période_construction et Méthode application DPE. Dans le cadre de notre analyse, il nous semble utile de garder l'année de construction et non la période de construction, la première variable étant plus précise. Nous pouvons également enlever la variable "Méthode_application_DPE".
Par ailleurs, nous allons sélectionner seulement les variables qui peuvent nous intéresser pour l'analyse. 

In [16]:
print(DPE_logements_existants["Méthode_application_DPE"].unique())
print(DPE_logements_existants["Cage_d'escalier"].unique())

['dpe appartement individuel'
 'dpe appartement généré à partir des données DPE immeuble'
 "dpe issu d'une étude thermique réglementaire RT2012 bâtiment : appartement"
 'dpe maison individuelle' 'dpe immeuble collectif'
 "dpe issu d'une étude thermique réglementaire RT2012 bâtiment : immeuble"]
[nan 'B3' 'NC' ... 'BTC' 'A8' '908']


In [18]:
DPE_logements_neufs["Année_construction"] = DPE_logements_neufs["Date_établissement_DPE"].astype(str).str[-4:]
DPE_logements_existants = DPE_logements_existants.drop(columns=["Période_construction", 
       "Méthode_application_DPE","Adresse_brute",
       "Nom__commune_(BAN)", "N°_voie_(BAN)",
       "Identifiant__BAN", "Code_postal_(BAN)",
       "Nom__rue_(BAN)","Code_postal_(brut)",
       "Cage_d'escalier",
       "Complément_d'adresse_logement",
       "Nom__commune_(Brut)", "N°_département_(BAN)",
       "Complément_d'adresse_bâtiment"])
DPE_logements_neufs = DPE_logements_neufs.drop(columns=["Adresse_brute",
       "Nom__commune_(BAN)", "N°_voie_(BAN)",
       "Identifiant__BAN", "Code_postal_(BAN)",
       "Nom__rue_(BAN)","Code_postal_(brut)",
       "Cage_d'escalier",
       "Complément_d'adresse_logement",
       "Nom__commune_(Brut)", "N°_département_(BAN)",
       "Complément_d'adresse_bâtiment"])

In [19]:
DPE_logements = pd.concat([DPE_logements_existants, DPE_logements_neufs], ignore_index=True)

In [20]:
DPE_logements['Année_construction'] = pd.to_numeric(DPE_logements['Année_construction'], errors='coerce')

In [21]:
outpath1 = 'DPE/DPE_après_2021.parquet'

s3.from_pandas_to_parquet_store_in_s3(DPE_logements, outpath1)

Fichier Parquet écrit avec succès dans DPE/DPE_après_2021.parquet


## 1.2. Valeurs foncières

### Téléchargement et fusion des bases

In [22]:
dtype_spec = {"Code commune": str}

path_valeursfoncieres_2022 = "Valeursfoncières/valeursfoncieres-2022.txt"
vf2022 = s3.read_txt_from_s3(path_valeursfoncieres_2022, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2021 = "Valeursfoncières/valeursfoncieres-2021.txt"
vf2021 = s3.read_txt_from_s3(path_valeursfoncieres_2021, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2020 = "Valeursfoncières/valeursfoncieres-2020.txt"
vf2020 = s3.read_txt_from_s3(path_valeursfoncieres_2020, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2019 = "Valeursfoncières/valeursfoncieres-2019.txt"
vf2019 = s3.read_txt_from_s3(path_valeursfoncieres_2019, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2018 = "Valeursfoncières/valeursfoncieres-2018.txt"
vf2018 = s3.read_txt_from_s3(path_valeursfoncieres_2018, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2017 = "Valeursfoncières/valeursfoncieres-2017.txt"
vf2017 = s3.read_txt_from_s3(path_valeursfoncieres_2017, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2016 = "Valeursfoncières/valeursfoncieres-2016.txt"
vf2016 = s3.read_txt_from_s3(path_valeursfoncieres_2016, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2015 = "Valeursfoncières/valeursfoncieres-2015.txt"
vf2015 = s3.read_txt_from_s3(path_valeursfoncieres_2015, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2014 = "Valeursfoncières/valeursfoncieres-2014.txt"
vf2014 = s3.read_txt_from_s3(path_valeursfoncieres_2014, sep="|", dtype_spec=dtype_spec)


In [23]:
vf = pd.concat([vf2022, vf2021, vf2020, vf2019, vf2018, vf2017, vf2016, vf2015, vf2014], ignore_index=True)


In [24]:
print(vf.columns)

Index(['Identifiant de document', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',
       'No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain', 'Code service CH'],
      dtype='object')


In [16]:
print(vf.dtypes)

Identifiant de document       float64
Reference document            float64
1 Articles CGI                float64
2 Articles CGI                float64
3 Articles CGI                float64
4 Articles CGI                float64
5 Articles CGI                float64
No disposition                  int64
Date mutation                  object
Nature mutation                object
Valeur fonciere                object
No voie                       float64
B/T/Q                          object
Type de voie                   object
Code voie                      object
Voie                           object
Code postal                   float64
Commune                        object
Code departement               object
Code commune                   object
Prefixe de section            float64
Section                        object
No plan                         int64
No Volume                      object
1er lot                        object
Surface Carrez du 1er lot      object
2eme lot    

In [17]:
print(vf["Nature mutation"].value_counts(dropna=False))
print(vf["Code voie"].value_counts(dropna=False))
print(vf["Section"].value_counts(dropna=False))
print(vf["Nature mutation"].value_counts(dropna=False))
print(vf["1er lot"].value_counts(dropna=False))

Nature mutation
Vente                                 27775517
Vente en l'état futur d'achèvement     2085142
Echange                                 405566
Adjudication                            111197
Vente terrain à bâtir                    96519
Expropriation                            41710
Name: count, dtype: int64
Code voie
NaN     257119
0020    139097
B005    137972
B003    135297
0040    132924
         ...  
X344         1
8947         1
V418         1
V812         1
V693         1
Name: count, Length: 20710, dtype: int64
Section
A     2241102
B     2119082
C     1494420
AB    1264471
D     1027006
       ...   
KQ         13
GR          7
WJ          6
SS          5
WG          4
Name: count, Length: 589, dtype: int64
Nature mutation
Vente                                 27775517
Vente en l'état futur d'achèvement     2085142
Echange                                 405566
Adjudication                            111197
Vente terrain à bâtir                    96519
Expropri

In [17]:
vf_clean = vf[['Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Voie', 'Code postal',
       'Commune', 'Code commune',
       'Surface Carrez du 1er lot', 'Surface Carrez du 2eme lot',
       'Surface Carrez du 3eme lot',
       'Surface Carrez du 4eme lot', 'Surface Carrez du 5eme lot',
       'Type local','Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain']]

In [18]:
print(vf_clean[vf_clean["Type local"] == "Appartement"].sample(3))

         Date mutation Nature mutation Valeur fonciere  No voie B/T/Q  \
21677754    03/05/2017           Vente       195000,00   3791.0   NaN   
27102458    31/12/2015           Vente             NaN      3.0   NaN   
8566187     27/03/2020           Vente       188000,00     60.0   NaN   

         Type de voie            Voie  Code postal    Commune Code commune  \
21677754          RTE  DE ST NAUPHARY      82000.0  MONTAUBAN          121   
27102458          RUE        PAUL PIC      69500.0       BRON           29   
8566187            AV    DE BRANCOLAR       6100.0       NICE           88   

          ... Surface Carrez du 2eme lot Surface Carrez du 3eme lot  \
21677754  ...                        NaN                        NaN   
27102458  ...                        NaN                        NaN   
8566187   ...                      41,18                        NaN   

         Surface Carrez du 4eme lot Surface Carrez du 5eme lot   Type local  \
21677754                      

In [19]:
outpath2 = 'Valeursfoncières/vf.parquet'

s3.from_pandas_to_parquet_store_in_s3(vf_clean, outpath2)

Fichier Parquet écrit avec succès dans Valeursfoncières/vf.parquet


### Normalisation des adresses

In [ ]:
# Convertir explicitement toutes les colonnes en chaînes
#vf['Adresse'] = vf['No voie'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
 #               vf['Type de voie'].fillna('').astype(str) + " " + \
 #               vf['Voie'].fillna('').astype(str) + ", " + \
 #               vf['Code postal'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
 #               vf['Commune'].fillna('').astype(str)
#print(vf.describe)

In [ ]:
#vf['Adresse'] = vf['Adresse'].str.strip().replace(r'^\s*$', None, regex=True)  # Supprime les adresses vides
#print(vf2022[['No voie', 'Type de voie', 'Voie', 'Code postal', 'Commune', 'Adresse']].head())

In [ ]:
#vf2022['Adresse_Normalisee'] = vf2022['Adresse'].apply(s3.normalize_address)

#print(vf2022[['Adresse', 'Adresse_Normalisee']].head())

TypeError: s3_connection.normalize_address() takes 1 positional argument but 2 were given